# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

AttributeError: 'World' object has no attribute 'Character'

For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [ ]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [ ]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

You can also _mask_ the terrain where the character has no knowledge. 

In [ ]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

## Analysis via list comprehension

Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [ ]:
[[person.name for person in t.population] for t in world.towns]

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [ ]:
[[(person.name,person.temperment) for person in t.population if person.temperment>.5] for t in world.towns]

You can also get summary stats on towns. For example, the average temperment per town.

In [ ]:
[(t.name,np.mean([person.temperment for person in t.population])) for t in world.towns]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [ ]:
w.get_people_where_char_has_visited(world)

In [ ]:
r = w.get_people_where_char_has_visited(world)[0]
r

In [ ]:
r['town'].nation

In [ ]:
w.get_relationships_node_map(world)

In [ ]:
where_the_char_has_been['nation'].dropna()

In [ ]:
nations_where_character_has_been = np.unique(where_the_char_has_been['nation'].dropna())
nations_where_character_has_been

You can use this to get lists of people who the character knows. 

In [ ]:
[[t for t in n.get_all_towns(world)] for n in world.nations if n.name in nations_where_character_has_been]

In [ ]:
[[t for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

In [ ]:
[(T.nation,T.name) for T in world.towns]

In [ ]:
[[t.get_person_data() for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

In [ ]:
masked_world.loc[world.Character.get_location_key()]

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [ ]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

## Getting the diplomacy and relationships data for the journal page

In [ ]:
rel = w.get_people_where_char_has_visited(world)
rel

In [ ]:
w.get_relationships_node_map(world)

In [ ]:
hierarchy = [{'nation':i.name,
              'children':i.diplomacy} for i in world.nations]



In [ ]:
p = rel[0]['town'].population[0]


In [ ]:
def get_people_where_char_has_visited(world):
    """
    returns a dictionary of {{"town":[<obj>],"people":[<obj>]}
    filtered to where the character has been. 
    """
    where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                    (world.df_features['terrain']=='town')].dropna()
    towns_and_people = [{"town":T,
                         "people":[t for t in T.population]} 
                        for T in world.towns if T.name in np.unique(w.where_the_char_has_been['feature'])]
    return towns_and_people

def get_relationships_node_map(world):
    nodes = [{'name':r['town'].name,
              'title':str(r['town']).split(":")[0],
              'nation':r['town'].diplomacy['nation'],
              'population':r['town'].pop,
              'type':r['town'].type,
              'location':r['town'].key,
              'founded year':r['town'].founded,
              'national fealty':r['town'].diplomacy['national fealty'],
             'children': [{'name':p.name,
                          'role':p.role,
                          'temperment':p.temperment,
                          'loyalty':p.loyalty,
                          'type':'person'} for p in r['people']]} 
             for r in w.get_people_where_char_has_visited(world)]
    return nodes
        
w.get_people_where_char_has_visited(world)

In [ ]:
relationships = w.get_relationships_node_map(world)
relationships

In [ ]:
nations = [s['nation'] for s in relationships]
nations

In [ ]:
[i for i in relationships]

In [ ]:
[{'nation':i,'towns':[t for t in relationships if t['nation']==i]} for i in nations]

In [ ]:
[i for i in nations]